<a href="https://colab.research.google.com/github/Nightmare125/Speech-Emotion-Recognition/blob/main/Speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>